In [ ]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE 
import cv2
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
!pip install patchify
image_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/CXR_png/")
mask_path = os.path.join("../input/chest-xray-masks-and-labels/Lung Segmentation/masks/")


In [ ]:
# we have 704 masks but 800 images. Hence we are going to
# make a 1-1 correspondance from mask to images, not the usual other way.
images = os.listdir(image_path)
mask = os.listdir(mask_path)
mask = [fName.split(".png")[0] for fName in mask]
image_file_name = [fName.split("_mask")[0] for fName in mask]

In [ ]:
check = [i for i in mask if "mask" in i]
print("Total mask that has modified name:",len(check))

In [ ]:
testing_files = set(os.listdir(image_path)) & set(os.listdir(mask_path))
training_files = check

def getData(X_shape, flag = "test"):
    im_array = []
    mask_array = []
    kernel = np.ones((3,3),np.float32)/9
    if flag == "test":
        for i in tqdm(testing_files): 
            im = cv2.resize(cv2.imread(os.path.join(image_path,i)),(X_shape,X_shape))[:,:,0]            
            im = cv2.filter2D(im,-1,kernel)
            mask = cv2.resize(cv2.imread(os.path.join(mask_path,i)),(X_shape,X_shape))[:,:,0]
            im_array.append(im)
            mask_array.append(mask)
        
        return im_array,mask_array
    
    if flag == "train":
        for i in tqdm(training_files): 
            im = cv2.resize(cv2.imread(os.path.join(image_path,i.split("_mask")[0]+".png")),(X_shape,X_shape))[:,:,0]
            im = cv2.filter2D(im,-1,kernel)
            mask = cv2.resize(cv2.imread(os.path.join(mask_path,i+".png")),(X_shape,X_shape))[:,:,0]

            im_array.append(im)
            mask_array.append(mask)

        return im_array,mask_array

In [ ]:
#perform sanity check

def plotMask(X,y):
    sample = []
    
    for i in range(6):
        left = X[i]
        right = y[i]
        combined = np.hstack((left,right))
        sample.append(combined)
        
        
    for i in range(0,6,3):

        plt.figure(figsize=(25,10))
        
        plt.subplot(2,3,1+i)
        plt.imshow(sample[i])
        
        plt.subplot(2,3,2+i)
        plt.imshow(sample[i+1])
        
        
        plt.subplot(2,3,3+i)
        plt.imshow(sample[i+2])
        
        plt.show()

In [ ]:
# Load training and testing data
dim = 512
print(dim)
X_train_1,y_train_1 = getData(dim,flag="train")
X, y = getData(dim)

#X = X + X_train_1
#y = y + y_train_1

In [ ]:
import patchify
import numpy as np
from sklearn.model_selection import train_test_split
import random


a_list = list(range(0, 138))
X_train = []
X_validate = []
X_test = []
y_train = []
y_validate = []
y_test = []

i = 0
while i < 80:
    temp = random.randint(0, len(a_list)-1)
    X_train.append(X[a_list[temp]])
    y_train.append(y[a_list[temp]])
    del a_list[temp]
    i = i+1

i = 0
while i < 20:
    temp = random.randint(0, len(a_list)-1)
    X_validate.append(X[a_list[temp]])
    y_validate.append(y[a_list[temp]])
    del a_list[temp]
    i = i + 1

i = 0
while i < 38:
    temp = random.randint(0, len(a_list)-1)
    X_test.append(X[a_list[temp]])
    y_test.append(y[a_list[temp]])
    del a_list[temp]
    i = i + 1

#create pachify arrays

X_patchify_train = []
Y_patchify_train = []
X_patchify_validate = []
Y_patchify_validate = []
X_patchify_test = []
Y_patchify_test = []

#create loop to patchify each image and its mask
    
for image in X_train:
    patches = patchify.patchify(image, (32, 32), step=32)
    X_patchify_train.append(patches)
    
for image in y_train:
    patches = patchify.patchify(image, (32, 32), step=32)
    Y_patchify_train.append(patches)
    
for image in X_validate:
    patches = patchify.patchify(image, (32, 32), step=32)
    X_patchify_validate.append(patches)
    
for image in y_validate:
    patches = patchify.patchify(image, (32, 32), step=32)
    Y_patchify_validate.append(patches)

for image in X_test:
    patches = patchify.patchify(image, (32, 32), step=32)
    X_patchify_test.append(patches)
    
for image in y_test:
    patches = patchify.patchify(image, (32, 32), step=32)
    Y_patchify_test.append(patches)

In [ ]:
Y_patchify_train_status = []
Y_patchify_validate_status = []
Y_patchify_test_status = []

X_patchify_train_updated = []
X_patchify_validate_updated = []
X_patchify_test_updated = []

train_ignore = []
validate_ignore = []
test_ignore = []


for patches in Y_patchify_train:
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            if np.mean(patches[i][j]) >= 51:
                Y_patchify_train_status.append((1,0))
                X_patchify_train_updated.append(patches[i][j]) 
            else:
                Y_patchify_train_status.append((0,1))
                X_patchify_train_updated.append(patches[i][j])
                

counter = 0
counter2 = 0
for patches in Y_patchify_validate:
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            if np.mean(patches[i][j]) >= 51:
                Y_patchify_validate_status.append((1,0))
                X_patchify_validate_updated.append(patches[i][j])
                
            else:
                Y_patchify_validate_status.append((0,1))
                X_patchify_validate_updated.append(patches[i][j])
                

print(counter)
print(counter2)

for patches in Y_patchify_test:
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            if np.mean(patches[i][j]) >= 51:
                Y_patchify_test_status.append((1,0))
                X_patchify_test_updated.append(patches[i][j])
            else:
                Y_patchify_test_status.append((0,1))
                X_patchify_test_updated.append(patches[i][j])

In [ ]:
print(len(Y_patchify_train_status))
print(len(X_patchify_train_updated))
print(len(Y_patchify_validate_status))
print(len(X_patchify_validate_updated))
print(len(Y_patchify_test_status))
print(len(X_patchify_test_updated))

In [ ]:
#nparrays for the patches
print(len(Y_patchify_train_status))
print(len(Y_patchify_test_status))
Y_patchify_train_status = np.array(Y_patchify_train_status).reshape(len(Y_patchify_train_status),2)
Y_patchify_validate_status = np.array(Y_patchify_validate_status).reshape(len(Y_patchify_validate_status),2)
Y_patchify_test_status = np.array(Y_patchify_test_status).reshape(len(Y_patchify_test_status),2)
#big number from taking total entries and dividing them by image size
X_patchify_train_updated = np.array(X_patchify_train_updated).reshape(len(X_patchify_train_updated),32,32,1)
X_patchify_test_updated = np.array(X_patchify_test_updated).reshape(len(X_patchify_test_updated),32,32,1)
X_patchify_validate_updated = np.array(X_patchify_validate_updated).reshape(len(X_patchify_validate_updated),32,32,1)

In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import os
import time

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(32,32,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.AUC()])



In [ ]:
loss_history = model.fit(x = X_patchify_train_updated,
                         y = Y_patchify_train_status,
                         batch_size = 128,
                         epochs = 20,
                         validation_data =(X_patchify_validate_updated,Y_patchify_validate_status))

In [ ]:
score, acc, *is_anything_else_being_returned = model.evaluate(X_patchify_test_updated,Y_patchify_test_status)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
#save model if needed
#model.save('./my_model')

In [ ]:
#!zip -r file.zip ./my_model

In [ ]:
#load model that was trained
#model = tf.keras.models.load_model(
#    '../input/automatic-lung-segmentation-model-part-1/my_model', custom_objects=None, compile=True, options=None
#)

import glob
covid_data = []
X_patchify_covid = []

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(5,5))


for file in glob.glob("../input/jpg-test/COVID-5.png"):
    print(file)
    img = cv2.resize(cv2.imread(file, cv2.IMREAD_GRAYSCALE), (512, 512))
    #img = clahe.apply(img)
    img = cv2.equalizeHist(img)
    covid_data.append(img)

for image in covid_data:
    patches = patchify.patchify(np.array(image), (32, 32), step=2)
    X_patchify_covid.append(patches)
    
X_patchify_covid = np.float16(np.array(X_patchify_covid).reshape(58081,32,32,1))

Y_patchify_covid = model.predict(X_patchify_covid)

results = []
counter = 0
counter2 = 0
for element in Y_patchify_covid:
    if element[0] > element[1]:
        results.append(1)
        counter += 1
    else:
        results.append(0)
        counter2 += 1
    
i = 0
j = 0
k = 0
counter = 0

covid_masks = []

count1 = 0
count2 = 0

print(len(covid_data))
while i < 1:
    j =0
    global temp
    temp = np.zeros((512, 512))
    while j < 512:
        k=0
        while k < 512:
            if j > 497 and (k < 260 and k > 240):
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
            elif k < 15 or j < 15 or k > 497 or j > 497:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
            elif results[counter] == 0:
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
                counter += 1
                
            else:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
                counter += 1
            k=k+2
        j=j+2
    covid_masks.append(temp)
    i = i + 1
    
#test maps
import matplotlib.pyplot as plt 
plt.imshow(covid_masks[0]) 
plt.show() 


In [ ]:
#create pachify arrays
X_patchify_train_2 = []
Y_patchify_train_2 = []
X_patchify_validate_2 = []
Y_patchify_validate_2 = []


for image in X_train:
    patches = patchify.patchify(image, (32, 32), step=2)
    X_patchify_train_2.append(patches)
    
for image in y_train:
    patches = patchify.patchify(image, (32, 32), step=2)
    Y_patchify_train_2.append(patches)
    
for image in X_validate:
    patches = patchify.patchify(image, (32, 32), step=2)
    X_patchify_validate_2.append(patches)
    
for image in y_validate:
    patches = patchify.patchify(image, (32, 32), step=2)
    Y_patchify_validate_2.append(patches)


In [ ]:
import gc
del score, acc, X_train, X_validate, X, y
del Y_patchify_train_status,Y_patchify_validate_status, Y_patchify_test_status, X_patchify_train_updated ,X_patchify_test_updated, X_patchify_validate_updated 
del Y_patchify_test, Y_patchify_validate, Y_patchify_train
gc.collect

In [ ]:
X_patchify_train_2 = (np.array(X_patchify_train_2).reshape(4646480,32,32,1))
Y_patchify_train_2_prediction = model.predict(X_patchify_train_2)

del X_patchify_train_2
gc.collect


X_patchify_validate_2 = (np.array(X_patchify_validate_2).reshape(1161620,32,32,1))
Y_patchify_validate_2_prediction = model.predict(X_patchify_validate_2)
del X_patchify_validate_2
gc.collect

test_result = []
train_result = []
validate_result = []

for element in Y_patchify_train_2_prediction:
    if element[0] > element[1]:
        train_result.append(1)
    else:
        train_result.append(0)

for element in Y_patchify_validate_2_prediction:
    if element[0] > element[1]:
        validate_result.append(1)
    else:
        validate_result.append(0)
        


In [ ]:
X_patchify_test_2 = []
Y_patchify_test_2 = []

for image in X_test:
    patches = patchify.patchify(image, (32, 32), step=2)
    X_patchify_test_2.append(patches)
    
for image in y_test:
    patches = patchify.patchify(image, (32, 32), step=2)
    Y_patchify_test_2.append(patches)

X_patchify_test_2 = (np.array(X_patchify_test_2).reshape(2207078,32,32,1))
Y_patchify_test_2_prediction = model.predict(X_patchify_test_2)

for element in Y_patchify_test_2_prediction:
    if element[0] > element[1]:
        test_result.append(1)
    else:
        test_result.append(0)

In [ ]:
del X_patchify_test_2
gc.collect

In [ ]:
i = 0
j = 0
k = 0

train_masks_2 = []

count1 = 0
count2 = 0
counter = 0

while i < 80:
    j = 0
    global temp
    temp = np.zeros((512, 512))
    while j < 512:
        k=0
        while k < 512:
            if j > 497 and (k < 260 and k > 240):
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
            elif k < 15 or j < 15 or k > 497 or j > 497:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
            elif train_result[counter] == 0:
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
                counter = counter + 1
            else:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
                counter = counter + 1
            k=k+2
        j=j+2
    train_masks_2.append(temp)
    i = i + 1

In [ ]:
i = 0
j = 0
k = 0

validate_masks_2 = []

count1 = 0
count2 = 0
counter = 0

while i < 20:
    j = 0
    global temp
    temp = np.zeros((512, 512))
    while j < 512:
        k=0
        while k < 512:
            if j > 497 and (k < 260 and k > 240):
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
            elif k < 15 or j < 15 or k > 497 or j > 497:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
            elif validate_result[counter] == 0:
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
                counter = counter + 1
            else:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
                counter = counter + 1
            k=k+2
        j=j+2
    validate_masks_2.append(temp)
    i = i + 1

In [ ]:
i = 0
j = 0
k = 0

test_masks_2 = []

count1 = 0
count2 = 0
counter = 0

while i < 38:
    j = 0
    global temp
    temp = np.zeros((512, 512))
    while j < 512:
        k=0
        while k < 512:
            if j > 497 and (k < 260 and k > 240):
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
            elif k < 15 or j < 15 or k > 497 or j > 497:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
            elif test_result[counter] == 0:
                temp[j][k] = 255
                temp[j-1][k] = 255
                temp[j][k-1] = 255
                temp[j-1][k-1] = 255
                counter = counter + 1
            else:
                temp[j][k] = 0
                temp[j-1][k] = 0
                temp[j][k-1] = 0
                temp[j-1][k-1] = 0
                counter = counter + 1
            k=k+2
        j=j+2
    test_masks_2.append(temp)
    i = i + 1

In [ ]:
#test maps
import matplotlib.pyplot as plt 
plt.imshow(train_masks_2[6]) 
plt.show() 

In [ ]:
from skimage.morphology import erosion, dilation, closing
import cv2 as cv
import scipy
#create masks for operations

cross = np.zeros((19,19))
i = 0
while i < 19:
    cross[9][i] = 1
    cross[i][9] = 1
    i = i + 1
    
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

circle_dilation = create_circular_mask(15,15)
circle_dilation = circle_dilation.astype(int)

circle_closing = create_circular_mask(19,19)
circle_closing = circle_closing.astype(int)

def get_contour_areas(contours):

    all_areas= []

    for cnt in contours:
        area= cv2.contourArea(cnt)
        all_areas.append(area)

    return all_areas

updated_test_masks = []
i = 0
while i < 38:

    img = erosion(test_masks_2[i], cross)
    img = dilation(img, circle_dilation)
    ret, thresh = cv.threshold(img, 127, 255, 0)
    thresh = thresh.astype(int)
    contours, hierarchy = cv.findContours(thresh, cv2.RETR_FLOODFILL, cv2.CHAIN_APPROX_SIMPLE)

    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
    temp = np.zeros((512,512), dtype=np.uint8)

    if cv2.pointPolygonTest(sorted_contours[0], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(1)
    elif cv2.pointPolygonTest(sorted_contours[2], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(2)
    else:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
        except:
            print(3)
    temp = closing(temp, circle_closing)
    updated_test_masks.append(temp)
    i = i + 1
    
    
updated_train_masks = []
i = 0
while i < 80:

    img = erosion(train_masks_2[i], cross)
    img = dilation(img, circle_dilation)
    ret, thresh = cv.threshold(img, 127, 255, 0)
    thresh = thresh.astype(int)
    contours, hierarchy = cv.findContours(thresh, cv2.RETR_FLOODFILL, cv2.CHAIN_APPROX_SIMPLE)

    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
    temp = np.zeros((512,512), dtype=np.uint8)

    if cv2.pointPolygonTest(sorted_contours[0], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(1)
    elif cv2.pointPolygonTest(sorted_contours[2], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(2)
    else:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
        except:
            print(3)

    
    temp = closing(temp, circle_closing)
    updated_train_masks.append(temp)
    i = i + 1
    
updated_validate_masks = []
i = 0
while i < 20:

    img = erosion(validate_masks_2[i], cross)
    img = dilation(img, circle_dilation)
    ret, thresh = cv.threshold(img, 127, 255, 0)
    thresh = thresh.astype(int)
    contours, hierarchy = cv.findContours(thresh, cv2.RETR_FLOODFILL, cv2.CHAIN_APPROX_SIMPLE)

    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)
    temp = np.zeros((512,512), dtype=np.uint8)

    if cv2.pointPolygonTest(sorted_contours[0], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(1)
    elif cv2.pointPolygonTest(sorted_contours[2], (256, 500), False) == 1:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[4]], color=(255,255,255))
        except:
            print(2)
    else:
        try:
            cv2.fillPoly(temp, pts = [sorted_contours[0]], color=(255,255,255))
            cv2.fillPoly(temp, pts = [sorted_contours[2]], color=(255,255,255))
        except:
            print(3)

    
    temp = closing(temp, circle_closing)
    updated_validate_masks.append(temp)
    i = i + 1

update_validate_masks_final = []
for image in updated_validate_masks:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    update_validate_masks_final.append(temp)

update_test_masks_final = []
for image in updated_test_masks:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    update_test_masks_final.append(temp)
    
update_train_masks_final = []
for image in updated_train_masks:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    update_train_masks_final.append(temp)

plt.imshow(temp)
print(i)

In [ ]:
y_train_small = []
y_validate_small = []
y_test_small = []

for image in y_train:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    y_train_small.append(temp)
for image in y_validate:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    y_validate_small.append(temp)
for image in y_test:
    temp = (cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC))
    temp = np.reshape(temp, (128, 128, 1))
    y_test_small.append(temp)

In [ ]:
"""
ResNet-18
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""


from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add, Reshape
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf


class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.max_pool = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")
        self.flat = Flatten()
        self.fc = Dense(256*256, activation="sigmoid")
        
        self.reshape = Reshape((256, 256, 1), input_shape=(65536,))
        self.max_pool_2 = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.max_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        out = self.reshape(out)
        out = self.max_pool_2(out)
        return out

In [ ]:
model = ResNet18(2)
model.build(input_shape = (None,128,128,1))
#use categorical_crossentropy since the label is one-hot encoded

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = opt,loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

In [ ]:
#copy y_test_small for later
Y_gt = y_test_small
update_train_masks_final = np.array(update_train_masks_final).reshape(80, 128,128, 1).astype("float32")/255
y_train_small = np.array(y_train_small).reshape(80, 128,128, 1).astype("float32")/255
update_validate_masks_final = np.array(update_validate_masks_final).reshape(20, 128,128, 1).astype("float32")/255
y_validate_small = np.array(y_validate_small).reshape(20, 128,128, 1).astype("float32")/255
update_test_masks_final = np.array(update_test_masks_final).reshape(38, 128,128, 1).astype("float32")/255
y_test_small = np.array(y_test_small).reshape(38, 128,128, 1).astype("float32")/255
print(update_train_masks_final.size)

In [ ]:
loss_history = model.fit(x = update_train_masks_final,
                         y = y_train_small,
                         batch_size = 2,
                         epochs = 30,
                         shuffle = True,
                         validation_data =(update_validate_masks_final,y_validate_small))

In [ ]:
score, acc, *is_anything_else_being_returned = model.evaluate(update_test_masks_final,y_test_small)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
import sklearn
from scipy import ndimage, misc
GT = Y_gt
initial_output = []
for i in range(len(update_test_masks_final)):
    initial_output.append(model.predict(update_test_masks_final[i].reshape(1,128, 128, 1)))
    
for i in range(len(initial_output)):
    initial_output[i] = initial_output[i] > 0.35

for i in range(len(initial_output)):
    initial_output[i] = ndimage.median_filter(initial_output[i], size=5)

initial_output = np.array(initial_output).reshape(622592)

GT = np.array(GT).reshape(622592) > 240

accurracy = sklearn.metrics.accuracy_score(GT,initial_output)
print("This is the accurracy output:")
print(accurracy)
precision = sklearn.metrics.precision_score(GT,initial_output)
print("This is the precision output:")
print(precision)
f1 = sklearn.metrics.f1_score(GT,initial_output)
print("This is the f1 score output:")
print(f1)
initial_output = np.array(initial_output).reshape(38, 128, 128, 1)
GT = (np.array(GT).reshape(38, 128, 128, 1))

In [ ]:
plt.imshow(Y_gt[3].reshape(128,128,1))

In [ ]:
x = model.predict(update_test_masks_final[3].reshape(1, 128, 128, 1))
plt.imshow(x.reshape(128,128,1))
print(np.min(x))

In [ ]:
plt.imshow(initial_output[3].reshape(128,128,1))
print(np.min(x))

In [ ]:
plt.imshow(y_test_small[1])
print(np.max(x))

In [ ]:
x = model.predict(update_test_masks_final[3].reshape(1, 128, 128, 1))
plt.imshow(x.reshape(128,128,1))
print(np.min(x))